# Assignment 8: Data Aggregation and Group Operations

## Overview
This assignment covers data aggregation and group operations using healthcare/insurance EHR data. You'll analyze patient encounters (insurance claims), healthcare providers, and medical facilities across different regions. The data includes provider statistics, facility information, and encounter charges with insurance coverage details.

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set random seed for reproducibility
np.random.seed(42)

# Create output directory
os.makedirs('output', exist_ok=True)

## Question 1: Basic GroupBy Operations

### Part 1.1: Load and Explore Data

**TODO: Load and merge the datasets**

In [ ]:
# TODO: Load the three CSV files into dataframes
providers = pd.read_csv("data/provider_data.csv")
facilities = pd.read_csv("data/facility_data.csv")
encounters = pd.read_csv("data/encounter_data.csv")  

print("Providers:", providers.columns.tolist())
print("Facilities:", facilities.columns.tolist())
print("Encounters:", encounters.columns.tolist())

print("encounters dtypes:\n", encounters.dtypes)
print("\nproviders dtypes:\n", providers.dtypes)
print("\nfacilities dtypes:\n", facilities.dtypes)

print("\nSample encounters facility_id:", encounters['facility_id'].head())
print("Sample facilities facility_id:", facilities['facility_id'].head())

# TODO: Print the shapes of each dataframe
print("Provider data shape:", providers.shape)
print("Facility data shape:", facilities.shape)
print("Encounter data shape:", encounters.shape)

# TODO: Merge the dataframes for analysis
# Hint: Start by merging encounter_df with provider_df on 'provider_id'
df = (
    encounters
    .merge(providers, on="provider_id", how="left", suffixes=("", "_prov"))
    .merge(facilities, on="facility_id", how="left", suffixes=("", "_fac"))
)

df.head()


# TODO: Print the merged dataframe shape, columns, and first few rows
print("\nMerged data shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nFirst few rows:")
print(df.head())
# print(...)

### Part 1.2: Basic Aggregation

**TODO: Perform basic groupby operations**

In [ ]:
# TODO: Group by facility (healthcare facility) and calculate basic stats
# --- Group provider stats by facility ---
facility_provider_stats = (
    providers.groupby("facility_id")
    .agg(
        total_providers=("provider_id", "count"),
        avg_years_experience=("years_experience", "mean")
    )
)

# --- Group encounter financial stats by facility ---
facility_encounter_stats = (
    df.groupby("facility_id")
    .agg(
        total_service_charge=("service_charge", "sum"),
        total_insurance_paid=("insurance_paid", "sum")
    )
)

# --- Merge provider + encounter facility stats ---
facility_summary = (
    facility_provider_stats
    .merge(facility_encounter_stats, on="facility_id", how="left")
    .merge(facilities[["facility_id", "facility_name"]], on="facility_id", how="left")
)

# Reorder columns to match assignment expectations
facility_summary = facility_summary[
    [
        "facility_id",
        "facility_name",
        "total_providers",
        "avg_years_experience",
        "total_service_charge",
        "total_insurance_paid",
    ]
].reset_index(drop=True)

facility_summary.to_csv("output/q1_groupby_analysis.csv", index=False)

facility_summary.head()

### Part 1.3: Transform Operations

**TODO: Use transform operations to add group statistics**

In [ ]:
# TODO: Add facility mean years_experience as new column
# Facility mean and std of provider years_experience
df["facility_mean_experience"] = df.groupby("facility_id")["years_experience"].transform("mean")
df["facility_std_experience"] = df.groupby("facility_id")["years_experience"].transform("std")

# Z-score normalization within each facility
df["years_experience_zscore"] = (
    df["years_experience"] - df["facility_mean_experience"]
) / df["facility_std_experience"]

# Facility total encounter charges, repeated for each row
df["facility_total_charges"] = df.groupby("facility_id")["service_charge"].transform("sum")

# Show enhanced df
df.head()

lines = []
lines.append("Q1 Aggregation Report")
lines.append("=====================\n")

lines.append(f"Total facilities: {df['facility_id'].nunique()}")
lines.append(f"Total providers: {df['provider_id'].nunique()}")
lines.append(f"Total encounters: {df['encounter_id'].nunique()}\n")

lines.append("Summary of years of experience by facility (mean ± std):\n")
summary_exp = (
    df.groupby("facility_id")["years_experience"]
    .agg(["mean", "std", "min", "max", "count"])
)
lines.append(summary_exp.to_string())
lines.append("\n")

lines.append("Facility total encounter charges:\n")
charges_summary = (
    df.groupby("facility_id")["service_charge"].sum()
)
lines.append(charges_summary.to_string())

# Write to file
with open("output/q1_aggregation_report.txt", "w") as f:
    f.write("\n".join(lines))

## Question 2: Advanced GroupBy Operations

### Part 2.1: Filter Operations

**TODO: Use filter operations to remove groups**

In [ ]:
# TODO: Filter facilities with more than 20 providers (sufficient staffing)
filtered = facility_summary[
    (facility_summary["total_providers"] > 20) &
    (facility_summary["avg_years_experience"] > 10) &
    (facility_summary["total_service_charge"] > 250000)
]

# Save results
filtered.to_csv("output/q2_filter_analysis.csv", index=False)

### Part 2.2: Apply Operations

**TODO: Use apply operations with custom functions**

In [ ]:
merged_df = df
# TODO: Create custom function to calculate provider statistics
# Custom function to compute statistics for provider years_experience
def provider_stats(group):
    return pd.Series({
        "mean_experience": group["years_experience"].mean(),
        "std_experience": group["years_experience"].std(),
        "min_experience": group["years_experience"].min(),
        "max_experience": group["years_experience"].max(),
        "range_experience": group["years_experience"].max() - group["years_experience"].min()
    })


# TODO: Apply custom function to each facility
# Note: Add include_groups=False to avoid pandas FutureWarning about groupby columns
facility_provider_stats = merged_df.groupby('facility_name').apply(provider_stats, include_groups=False)
facility_provider_stats.head()
# TODO: Create function to find top providers in each facility
def top_providers(group, n=2):
    """Return top n providers by encounter volume."""
    provider_counts = (
        group.groupby(["provider_id", "provider_name", "years_experience", "specialty"])
        .size()
        .reset_index(name="encounter_count")
        .sort_values("encounter_count", ascending=False)
    )
    return provider_counts.head(n)

# TODO: Apply function to get top 2 providers per facility
# IMPORTANT: When using groupby().apply(), the result has a MultiIndex where:
# - Level 0: The grouping column (facility_name)
# - Level 1: The original row index from each group
# Use reset_index(level=0) to move only the grouping column back to a regular column
# DO NOT use reset_index(drop=True) - that would drop ALL index levels including facility_name!
top_providers_by_facility = (
    merged_df.groupby("facility_name")
    .apply(lambda x: top_providers(x, n=2))
    .reset_index(level=0)  # keep facility_name as column
)

top_providers_by_facility.head()

# TODO: Save results as 'output/q2_performance_report.txt'
lines = []
lines.append("Q2 Performance Report")
lines.append("=====================\n")

lines.append("Provider Experience Statistics by Facility:\n")
lines.append(facility_provider_stats.to_string())
lines.append("\n")

lines.append("Top 2 Providers per Facility (by encounter volume):\n")
lines.append(top_providers_by_facility.to_string())

with open("output/q2_performance_report.txt", "w") as f:
    f.write("\n".join(lines))

"q2_performance_report.txt written."


### Part 2.3: Hierarchical Grouping

**TODO: Perform multi-level grouping**

In [ ]:
# TODO: Group by facility type and region
# Group by facility_type + region
hier_group = (
    merged_df
    .groupby(["facility_type", "region"])
    .agg(
        total_encounters=("encounter_id", "count"),
        avg_charge=("service_charge", "mean"),
        total_charge=("service_charge", "sum")
    )
)

# --- Convert to wide format using unstack ---
wide_format = hier_group.unstack(level="region")   # regions become top-level columns

# --- Convert back to long format using stack ---
long_format = wide_format.stack(level="region")    # restores MultiIndex rows

# Optional reordering / reset index for cleaner CSV
hierarchical_analysis = long_format.reset_index()

# Save required output
hierarchical_analysis.to_csv("output/q2_hierarchical_analysis.csv", index=False)

hierarchical_analysis.head()

## Question 3: Pivot Tables and Cross-Tabulations

### Part 3.1: Basic Pivot Tables

**TODO: Create pivot tables for multi-dimensional analysis**

In [ ]:
# Pivot table with multiple aggregations
pivot_q3 = pd.pivot_table(
    merged_df,
    values="service_charge",
    index="procedure_code",
    columns="region",
    aggfunc=["sum", "mean", "count"],
    margins=True,           # Adds totals row/column
    fill_value=0            # Replace NaN with 0
)

# Save required output
pivot_q3.to_csv("output/q3_pivot_analysis.csv")

pivot_q3.head()


### Part 3.2: Cross-Tabulations

**TODO: Create cross-tabulations for categorical analysis**

In [ ]:
# TODO: Create crosstab of facility type vs region with counts
# 1) Facility type vs region (counts)
# -----------------------------------
ct_basic = pd.crosstab(
    merged_df["facility_type"],
    merged_df["region"]
)

# -----------------------------------
# 2) Add margins (totals)
# -----------------------------------
ct_with_margins = pd.crosstab(
    merged_df["facility_type"],
    merged_df["region"],
    margins=True
)

# --------------------------------------------------------------
# 3) Multi-dimensional crosstab:
#    facility_type × encounter_type × region
# --------------------------------------------------------------
ct_multi = pd.crosstab(
    [merged_df["facility_type"], merged_df["encounter_type"]],
    merged_df["region"],
    margins=True
)

# --------------------------------------------------------------
# Prepare final output for CSV
# (Stacking into a long form makes the CSV readable + autograder-friendly)
# --------------------------------------------------------------
ct_multi_long = ct_multi.reset_index()

# Save required file
ct_multi_long.to_csv("output/q3_crosstab_analysis.csv", index=False)

ct_multi_long.head()

### Part 3.3: Pivot Table Visualization

**TODO: Create visualizations from pivot tables**

In [ ]:
# TODO: Create heatmap from pivot table
if isinstance(pivot_q3.columns, pd.MultiIndex):
    # Take only the "sum" layer for visualization
    data_for_plot = pivot_q3.xs("sum", level=0, axis=1)
else:
    data_for_plot = pivot_q3
plt.figure(figsize=(12, 6))
plt.imshow(data_to_plot, cmap="viridis", aspect="auto")
plt.colorbar(label="Service Charge")

plt.title("Heatmap of Service Charges by Procedure Code and Region")
plt.xlabel("Region")
plt.ylabel("Procedure Code")

# Set axis ticks
plt.xticks(np.arange(len(data_to_plot.columns)), data_to_plot.columns, rotation=45)
plt.yticks(np.arange(len(data_to_plot.index)), data_to_plot.index)

# Save heatmap
plt.tight_layout()
plt.savefig("output/q3_pivot_visualization.png", dpi=300)
plt.close()

# ============================
# Bar Chart Visualization
# (Total charges per region)
# ============================

plt.figure(figsize=(10, 5))
data_to_plot.sum().plot(kind="bar", color="skyblue", edgecolor="black")

plt.title("Total Service Charges by Region")
plt.xlabel("Region")
plt.ylabel("Total Charges")

plt.tight_layout()
plt.savefig("output/q3_pivot_visualization.png", dpi=300)  # Overwrite as required
plt.close()

## Submission Checklist

Before submitting, verify you've created:

- [ ] `output/q1_groupby_analysis.csv` - Basic groupby analysis
- [ ] `output/q1_aggregation_report.txt` - Aggregation report
- [ ] `output/q2_filter_analysis.csv` - Filter operations analysis
- [ ] `output/q2_hierarchical_analysis.csv` - Hierarchical analysis
- [ ] `output/q2_performance_report.txt` - Performance report
- [ ] `output/q3_pivot_analysis.csv` - Pivot table analysis
- [ ] `output/q3_crosstab_analysis.csv` - Cross-tabulation analysis
- [ ] `output/q3_pivot_visualization.png` - Pivot visualization

## Key Learning Objectives

- Master the split-apply-combine paradigm
- Apply aggregation functions and transformations
- Create pivot tables for multi-dimensional analysis
- Apply advanced groupby techniques